In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Environment setup complete")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Environment setup complete
Pandas version: 2.3.1
NumPy version: 1.26.4


In [2]:
# Load the dataset
df = pd.read_csv('accepted_2007_to_2018Q4.csv', low_memory=False)

print(f"Dataset shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Check if issue_d exists and has valid dates
if 'issue_d' in df.columns:
    df['issue_d'] = pd.to_datetime(df['issue_d'], errors='coerce')
    valid_dates = df['issue_d'].notna()
    if valid_dates.any():
        print(f"\nDate range: {df.loc[valid_dates, 'issue_d'].min()} to {df.loc[valid_dates, 'issue_d'].max()}")
    else:
        print("\nNo valid dates found in issue_d column")
else:
    print("\nissue_d column not found in dataset")

Dataset shape: 2,260,701 rows x 151 columns

Memory usage: 5992.29 MB

Date range: 2007-06-01 00:00:00 to 2018-12-01 00:00:00


In [3]:
# Examine loan status distribution
print("Loan Status Distribution:")
print(df['loan_status'].value_counts())
print("\nPercentage distribution:")
print(df['loan_status'].value_counts(normalize=True) * 100)

Loan Status Distribution:
loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64

Percentage distribution:
loan_status
Fully Paid                                            47.630
Current                                               38.852
Charged Off                                           11.880
Late (31-120 days)                                     0.950
In Grace Period                                        0.373
Late (16-30 days)         

In [4]:
# Define default categories based on lending industry standards
# Charged Off, Default, Does not meet credit policy (Charged Off) = Bad loans
# Fully Paid, Current, Does not meet credit policy (Fully Paid) = Good loans
# Late, In Grace Period = Exclude (insufficient outcome data)

default_categories = [
    'Charged Off',
    'Default',
    'Does not meet the credit policy. Status:Charged Off'
]

paid_categories = [
    'Fully Paid',
    'Does not meet the credit policy. Status:Fully Paid'
]

# Filter to loans with known outcomes
df_complete = df[df['loan_status'].isin(default_categories + paid_categories)].copy()

# Create binary target
df_complete['is_default'] = df_complete['loan_status'].isin(default_categories).astype(int)

print(f"Original dataset: {len(df):,} loans")
print(f"Complete outcomes: {len(df_complete):,} loans")
print(f"Excluded loans: {len(df) - len(df_complete):,} (ongoing or late)")
print(f"\nDefault rate: {df_complete['is_default'].mean()*100:.2f}%")

Original dataset: 2,260,701 loans
Complete outcomes: 1,348,099 loans
Excluded loans: 912,602 (ongoing or late)

Default rate: 19.98%


In [14]:
# Calculate actual financial losses in the dataset
df_complete['funded_amnt'] = pd.to_numeric(df_complete['funded_amnt'], errors='coerce')
df_complete['total_pymnt'] = pd.to_numeric(df_complete['total_pymnt'], errors='coerce')
df_complete['recoveries'] = pd.to_numeric(df_complete['recoveries'], errors='coerce')

# Net loss per loan
df_complete['net_loss'] = np.where(
    df_complete['is_default'] == 1,
    df_complete['funded_amnt'] - df_complete['total_pymnt'] - df_complete['recoveries'],
    0
)

# Summary statistics
total_funded = df_complete['funded_amnt'].sum()
total_defaults = df_complete['is_default'].sum()
total_loss = df_complete['net_loss'].sum()
avg_loss_per_default = df_complete[df_complete['is_default']==1]['net_loss'].mean()
current_default_rate = df_complete['is_default'].mean()

print("FINANCIAL IMPACT ANALYSIS")
print("="*60)
print(f"Total amount funded: ${total_funded:,.0f}")
print(f"Total number of defaults: {total_defaults:,}")
print(f"Total losses from defaults: ${total_loss:,.0f}")
print(f"Average loss per default: ${avg_loss_per_default:,.0f}")
print(f"Loss rate (as % of funded): {(total_loss/total_funded)*100:.2f}%")
print(f"\nCurrent default rate: {current_default_rate*100:.2f}%")

# Correct savings calculation
# If we reduce default rate by 1%, we prevent (total_loans * 0.01) defaults
total_loans = len(df_complete)
one_pct_reduction_savings = total_loans * 0.01 * current_default_rate * avg_loss_per_default
two_pct_reduction_savings = total_loans * 0.02 * current_default_rate * avg_loss_per_default

print("\nBusiness Case:")
print(f"A 1 percentage point reduction in default rate would save: ${one_pct_reduction_savings:,.0f}")
print(f"A 2 percentage point reduction in default rate would save: ${two_pct_reduction_savings:,.0f}")

FINANCIAL IMPACT ANALYSIS
Total amount funded: $19,412,877,700
Total number of defaults: 269,360
Total losses from defaults: $1,629,691,615
Average loss per default: $6,050
Loss rate (as % of funded): 8.39%

Current default rate: 19.98%

Business Case:
A 1 percentage point reduction in default rate would save: $16,296,916
A 2 percentage point reduction in default rate would save: $32,593,832


In [16]:
# Convert issue date to datetime
df_complete['issue_d'] = pd.to_datetime(df_complete['issue_d'])
df_complete['issue_year'] = df_complete['issue_d'].dt.year

# Calculate annual default rates
annual_defaults = df_complete.groupby('issue_year').agg({
    'is_default': ['sum', 'mean', 'count'],
    'funded_amnt': 'sum',
    'net_loss': 'sum'
}).round(4)

annual_defaults.columns = ['Total_Defaults', 'Default_Rate', 'Total_Loans', 
                           'Total_Funded', 'Total_Loss']

print("ANNUAL PERFORMANCE TRENDS")
print("="*60)
print(annual_defaults)

ANNUAL PERFORMANCE TRENDS
            Total_Defaults  Default_Rate  Total_Loans   Total_Funded  \
issue_year                                                             
2007                   158         0.262          603    4791550.000   
2008                   496         0.207         2393   19975025.000   
2009                   723         0.137         5281   51814750.000   
2010                  1757         0.140        12537  126351175.000   
2011                  3297         0.152        21721  257363650.000   
2012                  8644         0.162        53367  717942625.000   
2013                 21024         0.156       134804 1982607275.000   
2014                 41162         0.184       223103 3253489500.000   
2015                 75804         0.202       375546 5498601150.000   
2016                 68252         0.233       293105 4240361600.000   
2017                 39169         0.231       169321 2421502475.000   
2018                  8874         0.1

In [18]:
# Analyze default rates by loan grade (A, B, C, D, E, F, G)
grade_analysis = df_complete.groupby('grade').agg({
    'is_default': ['count', 'sum', 'mean'],
    'funded_amnt': ['mean', 'sum'],
    'int_rate': 'mean',
    'net_loss': 'sum'
}).round(4)

grade_analysis.columns = ['Total_Loans', 'Defaults', 'Default_Rate', 
                          'Avg_Loan_Amount', 'Total_Funded', 
                          'Avg_Interest_Rate', 'Total_Loss']

print("DEFAULT RATES BY LOAN GRADE")
print("="*60)
print(grade_analysis)
print("\nKey Insight: Higher grades (A) should have lower default rates than lower grades (F,G)")

DEFAULT RATES BY LOAN GRADE
       Total_Loans  Defaults  Default_Rate  Avg_Loan_Amount   Total_Funded  \
grade                                                                        
A           235193     14214         0.060        13879.205 3264291775.000   
B           393102     52661         0.134        13225.061 5198798075.000   
C           382323     85805         0.224        14174.676 5419304775.000   
D           201657     61264         0.304        15242.683 3073793625.000   
E            94192     36199         0.384        17545.589 1652654100.000   
F            32306     14585         0.452        19003.804  613936900.000   
G             9326      4632         0.497        20383.707  190098450.000   

       Avg_Interest_Rate    Total_Loss  
grade                                   
A                  7.114  77303614.031  
B                 10.679 279318927.228  
C                 14.019 504017883.467  
D                 17.710 382583945.928  
E                 21.10

In [20]:
# Document the business objectives
business_objectives = {
    'Primary Objective': 'Reduce default rate while maintaining approval volume',
    'Target Default Reduction': '15-20% relative reduction',
    'Minimum Approval Rate': '70% of current volume',
    'Risk Tolerance': 'Accept up to 5% false positive rate (good customers rejected)',
    'Model Performance Target': 'AUC > 0.70, Precision > 0.60 for default class'
}

constraints = {
    'Regulatory': 'Must provide explanations for denials (FCRA compliance)',
    'Fairness': 'No disparate impact on protected classes',
    'Operational': 'Predictions must be available within 2 seconds',
    'Business': 'Model must be interpretable for credit officers'
}

print("BUSINESS OBJECTIVES")
print("="*60)
for key, value in business_objectives.items():
    print(f"{key}: {value}")

print("\nCONSTRAINTS")
print("="*60)
for key, value in constraints.items():
    print(f"{key}: {value}")

BUSINESS OBJECTIVES
Primary Objective: Reduce default rate while maintaining approval volume
Target Default Reduction: 15-20% relative reduction
Minimum Approval Rate: 70% of current volume
Risk Tolerance: Accept up to 5% false positive rate (good customers rejected)
Model Performance Target: AUC > 0.70, Precision > 0.60 for default class

CONSTRAINTS
Regulatory: Must provide explanations for denials (FCRA compliance)
Fairness: No disparate impact on protected classes
Operational: Predictions must be available within 2 seconds
Business: Model must be interpretable for credit officers


In [35]:
# Save the filtered dataset with target variable
import os

# Create directory if it doesn't exist
os.makedirs('../data/processed', exist_ok=True)

# Save with error handling
try:
    df_complete.to_csv('accepted_2007_to_2018Q4.csv', index=False)
    print(f"Processed dataset saved successfully: {len(df_complete):,} rows")
except Exception as e:
    print(f"Could not save to CSV due to: {str(e)}")
    print("Attempting to save as pickle instead...")
    df_complete.to_pickle('../data/processed/loans_with_target.pkl')
    print("Saved as pickle file instead")

print(f"Features: {df_complete.shape[1]} columns")
print(f"Target variable: is_default")
print(f"Default rate: {df_complete['is_default'].mean()*100:.2f}%")

Processed dataset saved successfully: 1,348,099 rows
Features: 154 columns
Target variable: is_default
Default rate: 19.98%


In [31]:
# Create summary statistics for executive presentation
summary_stats = {
    'total_loans_analyzed': len(df_complete),
    'default_rate_percent': round(df_complete['is_default'].mean() * 100, 2),
    'total_loss_amount': round(total_loss, 0),
    'avg_loss_per_default': round(avg_loss_per_default, 0),
    'potential_annual_savings_2pct_reduction': round(total_funded * 0.02 * avg_loss_per_default / 11, 0),
    'date_range': f"{df_complete['issue_d'].min().date()} to {df_complete['issue_d'].max().date()}"
}

print("EXECUTIVE SUMMARY STATISTICS")
print("="*60)
for key, value in summary_stats.items():
    print(f"{key}: {value}")
    
# Create reports directory and save
import os
os.makedirs('../reports', exist_ok=True)

import json
with open('../reports/business_metrics.json', 'w') as f:
    json.dump(summary_stats, f, indent=4, default=str)
    
print("\nMetrics saved to reports/business_metrics.json")

EXECUTIVE SUMMARY STATISTICS
total_loans_analyzed: 1348099
default_rate_percent: 19.98
total_loss_amount: 1629691615.0
avg_loss_per_default: 6050.0
potential_annual_savings_2pct_reduction: 213549990657.0
date_range: 2007-06-01 to 2018-12-01

Metrics saved to reports/business_metrics.json
